In [ ]:
import torch
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset
import fastai.tabular.all as ft
from fastai.callback.tracker import EarlyStoppingCallback

#decodificar un cromosoma binario a hiperparametros
def decode_chromosome(chrom):
    # cada variable representa cad hiperparemetro
    seq_len_bits = chrom[0:6] #bits para longitud
    hidden_bits = chrom[6:8] #bits para tamaño de la capa oculta
    dropout_bits = chrom[8:14] #para dropout
    lr_bits = chrom[14:24] # bits para learning rate
    batch_bits = chrom[24:26] # bits para tamaño del batch

    # convertir bits a número entero
    seq_len = int(seq_len_bits, 2) % 41 + 20 # hacer el mapeo al rango

    # para tamaño del hidden
    hidden_opts = [32, 64, 128]
    hidden_size = hidden_opts[int(hidden_bits, 2) % 3]

    # convertir bits a valor decimla entre 0-0.4
    dropout = int(dropout_bits, 2) / 100
    dropout = min(dropout, 0.4)

    # bits a escala logaritmica
    lr = 10 ** (-4 + int(lr_bits, 2) / 1023 * 2)

    # para tamaño del batch
    batch_opts = [32, 64, 128]
    batch_size = batch_opts[int(batch_bits, 2) % 3]

    #diccionario con los hiperparametros decodificados
    return {
        'seq_len': seq_len,
        'hidden_size': hidden_size,
        'dropout': dropout,
        'lr': lr,
        'batch_size': batch_size
    }


# fitness de un cromosoma
def fitness(chromosome):
    try:
        # decodifando hiperparametros del cromosoma
        params = decode_chromosome(chromosome)
        seq_len = params['seq_len']
        hidden_size = params['hidden_size']
        dropout = params['dropout']
        lr = params['lr']
        batch_size = params['batch_size']

        # train_ds = BTC_ds(T[:-1000], seq_len)
        # valid_ds = BTC_ds(T[-1000 - seq_len:], seq_len)

        # checar que el tamaño del dataset sea suficiente para el batch
        if len(train_ds) < batch_size or len(valid_ds) < batch_size:
            return 1e6

        dls = ft.DataLoaders.from_dsets(train_ds, valid_ds, bs=batch_size, device='cuda:0')
        model = BitcoinLSTM(input_size=1, hidden_size=hidden_size, num_layers=2, dropout=dropout)

        learn = ft.Learner(
            dls,
            model,
            loss_func=ft.MSELossFlat(),
            opt_func=ft.ranger,
            cbs=EarlyStoppingCallback(monitor='valid_loss', patience=3)
        )

        learn.fit_one_cycle(10, lr)
        val_loss = learn.validate()[0]
        print(f"Params: {params}, Val Loss: {val_loss:.6f}")
        return val_loss.item()

    except Exception as e:
        # si hay error, devolver un valor de fitness muy malo
        print(f"Error: {e}")
        return 1e6